In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# RQ1: Which technologies have been investigated in the last decade?

In [3]:
#Datene einlesen
#data = pd.read_csv('./data/data.csv')
#data.drop(["Unnamed: 0"], axis = 1, inplace = True)

data = pd.read_csv('./data/export1.csv',dtype={'Abstract': str}, usecols = ['Document Title','Abstract','Publication Year'])
data.head()

,Document Title,Publication Year,Abstract
0,What Do We Know about Knowledge Management? Pr...,2009,There have been many claims about knowledge ma...
1,Research on National and International Softwar...,2009,The article analyzed the basic life cycle mode...
2,Advancing Software Engineering Professional Ed...,2011,The paper mentions that a reference curriculum...
3,Putting Human Aspects of Software Engineering ...,2010,Although people-related issues are central fac...
4,Where's the Theory for Software Engineering?,2012,"Darwin's theory of natural selection, Maxwell'..."


In [4]:
data.columns

Index(['Document Title', 'Publication Year', 'Abstract'], dtype='object')

In [5]:
data.count()

Document Title      1946
Publication Year    1946
Abstract            1933
dtype: int64

In [6]:
def CreateTermDocMatrix(column):
        
    count_vectorizer = CountVectorizer(stop_words='english',ngram_range = (1,2),dtype=np.int32)
    sparse_matrix = count_vectorizer.fit_transform(data[column].values.astype('U'))

    doc_term_matrix = sparse_matrix.todense()
    df = pd.DataFrame(doc_term_matrix, columns=count_vectorizer.get_feature_names(), dtype=np.int32)
    df['PublishingYear'] = data['Publication Year']
    returndf = df.groupby('PublishingYear').sum().transpose().astype('int32')
    return returndf

In [7]:
def ShowWord(data,word):
    d = data.loc[word]
    d.plot.bar()

In [8]:
def SpaltenEntf(data):
    y = []
    for item in data.columns:
        y = data.columns


    for index in range(len(y)):
        x = y[index]
        if x in stopwords or x.isnumeric():
            del data[y[index]]
    return data

In [9]:
#Untersuchung Abstract
#abstract = CreateTermDocMatrix('Abstract')

In [10]:
#b = abstract.loc[abstract[2011] > 20] 

In [11]:
#b.plot.bar()
#print(b)

In [12]:
from collections import Counter
#cnt = Counter()

#for abstract in data['Abstract']:
#    if type(abstract) is str:
#        wordlist = abstract.split(' ')
#        for word in wordlist:
#            cnt[word] += 1
        
#cnt.most_common(30)

In [13]:
#for abstract in data.itertuples():
#    print(abstract[1])
#    print(abstract[2])
#    print(abstract[3])

In [14]:
data = pd.read_csv('./data/data.csv',usecols = ['Document Title','Abstract','Publication Year'], nrows =1000)
data.columns = ['title','abstract','year']

In [15]:
#Untersuchung Title
#title = CreateTermDocMatrix('title')

#print(title)
#title = SpaltenEntf(title)
#print(title)

#b = title.loc[title[2017] > 5] 

#b.plot.bar()
#print(b)
#b.shape

In [16]:
#import spacy
#from spacy.symbols import NOUN
#from collections import Counter


#nlp = spacy.load("en_core_web_sm")

#cnt = Counter()
#tokenlist = []
#i = 1

#for columns in data.itertuples():

#    text = columns[1] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
#    if type(text) is str:
#        doc = nlp(text)

#        for noun_phrase in list(doc.noun_chunks):
#             noun_phrase.merge(noun_phrase.root.tag_, noun_phrase.root.lemma_, noun_phrase.root.ent_type_)

#        [(token.text,token.pos_) for token in doc]

In [17]:
from matplotlib import pyplot as plt
years = []

#Balkendiagramm
#plt.bar(range(len(years)),anzah)

#Titel
#plt.title("Toller Plot")
#plt.ylabel("# of awards")

#plt.xticks(range(len(years)),years)
#plt.show()

In [18]:
#pattern = [{'POS': 'ADJ', 'OP': '?'},
 #          {'OP': '+', 'POS': 'NOUN'},
  #         {'POS': 'NOUN', 'OP': '?'}]

#doc = nlp("I work on artificial intelligence topics.")

#matcher.add("match_nouns", [pattern])
#matches = matcher(doc)
#sp  = [doc[match[1]:match[2]] for match in matches] 

#filtered = filter_spans(sp)
#with doc.retokenize() as retokenizer:
#    for s in filtered:
#        retokenizer.merge(s, attrs = {"POS": "NOUN"})
#[(token.pos_, token.text) for token in doc]

#doc[3].vector == (nlp.vocab["artificial"].vector + 
#                  nlp.vocab["intelligence"].vector + 
#                  nlp.vocab["topics"].vector)/3

In [19]:
import spacy
from spacy.symbols import NOUN
from collections import Counter


nlp = spacy.load("en_core_web_sm")

words = Counter()
words_chunk = Counter()
tokenlist = []
chunklist = []
i = 1

for columns in data.itertuples():

    text = columns[3] #1 = title 2 = year 3 = abstract
    #print(type(abstract))
    # Verarbeite den Text
    if type(text) is str:
        doc = nlp(text)
        #list(dosc.noun_chunks)
        #print(i)
        i+= 1
        

        
        for token in doc:
            # Greife auf den Text, die Wortart und die Dependenzrelation des Tokens zu
            token_text = token.text
            token_pos = token.pos_
            token_dep = token.dep_
            token_ent = token.ent_type_
            
            if token.pos == NOUN:
                if not token.is_stop:
                    #print(f"{token_text}: {token_ent}")
                    #print(token.tag_)
                    words[token_text] += 1
                    #tokenlist.append([token_text,columns[2],token_ent])
                    
        for chunk in doc.noun_chunks:
            words_chunk[chunk.text] += 1


#tokenlist

In [20]:
words.most_common(100)

[('paper', 741),
 ('model', 635),
 ('system', 618),
 ('method', 600),
 ('software', 554),
 ('algorithm', 478),
 ('data', 381),
 ('results', 369),
 ('time', 281),
 ('process', 263),
 ('network', 260),
 ('information', 258),
 ('analysis', 237),
 ('problem', 234),
 ('development', 232),
 ('performance', 203),
 ('approach', 197),
 ('design', 184),
 ('engineering', 179),
 ('methods', 179),
 ('systems', 175),
 ('application', 171),
 ('quality', 170),
 ('image', 152),
 ('simulation', 147),
 ('technology', 145),
 ('control', 143),
 ('order', 143),
 ('research', 138),
 ('service', 130),
 ('knowledge', 128),
 ('test', 128),
 ('structure', 127),
 ('study', 123),
 ('management', 118),
 ('optimization', 110),
 ('testing', 107),
 ('experiment', 107),
 ('models', 105),
 ('function', 105),
 ('Web', 101),
 ('theory', 99),
 ('teaching', 99),
 ('evaluation', 99),
 ('state', 98),
 ('problems', 96),
 ('parameters', 96),
 ('framework', 95),
 ('set', 94),
 ('number', 92),
 ('detection', 91),
 ('level', 90),


In [21]:
words_chunk.most_common(100)

[('we', 430),
 ('this paper', 379),
 ('it', 319),
 ('This paper', 219),
 ('We', 211),
 ('It', 179),
 ('order', 116),
 ('they', 80),
 ('the system', 60),
 ('them', 57),
 ('-', 55),
 ('the paper', 55),
 ('the model', 54),
 ('algorithm', 54),
 ('The results', 53),
 ('the method', 47),
 ('The paper', 43),
 ('data', 43),
 ('the performance', 41),
 ('This article', 40),
 ('students', 39),
 ('the process', 39),
 ('the problem', 39),
 ('this method', 37),
 ('software engineering', 36),
 ('users', 34),
 ('the effectiveness', 34),
 ('the algorithm', 33),
 ('Experimental results', 33),
 ('the application', 32),
 ('the results', 31),
 ('developers', 31),
 ('the efficiency', 31),
 ('software', 30),
 ('terms', 29),
 ('this problem', 29),
 ('analysis', 29),
 ('China', 28),
 ('the basis', 28),
 ('the quality', 28),
 ('a set', 26),
 ('The experimental results', 26),
 ('the design', 26),
 ('what', 25),
 ('the same time', 25),
 ('development', 24),
 ('practice', 24),
 ('time', 24),
 ('the analysis', 24),